# 事業報告書生成
まずは以下のページに進む

[事業報告ページ](https://www.minpaku.mlit.go.jp/jigyo/CustomerHomePage)

TODO:
- ~~次回は同じファイルで複数の登録番号の記録を提出してみる~~ 完了
    - 宿泊履歴が無いとき、mode='all'でやるとエラーになる
    - mode='separate'だと問題ない
- 宿泊履歴があるときにどうなるかを確認する

In [2]:
import datetime
from collections import defaultdict
import os
import csv
from src.constants import REG_NUM_PAIR

In [3]:
# 期間(月)　04~05, 06~07, 08~09, 10~11, 12~01
csv_head = '届出番号,報告期間,宿泊日数,宿泊者数,宿泊延べ人数,日本,韓国,台湾,香港,中国,タイ,シンガポール,マレーシア,インドネシア,フィリピン,ベトナム,インド,英国,ドイツ,フランス,イタリア,スペイン,ロシア,米国,カナダ,オーストラリア,その他,宿泊日'
line_format = '{regist_number},{period},{days},{number_of_resident},{amount},{japan},{korea},{taiwan},{hongkong},{china},{thai},{singapore},{malaysia},{indonesia},{philippine},{vietnam},{india},{uk},{germany},{france},{italy},{spain},{russia},{usa},{canada},{australia},{other},{dates}'


In [4]:
def read_data(path='./data_to_feed.txt'):
    'to read the data file'
    with open(path) as f:
        res = [i.rstrip('\n') for i in f.readlines() if i[0] != '#' and len(i)>4]
    return res

def parse_input(s:str)->dict:
    '''parse a string that have poor data, and return a dict
    '''
    s = s.rstrip('\n').split(',')
    data = {
        'regist_number':REG_NUM_PAIR[s[0]], # '届け出番号':'番号'
        'country':s[4],# '居住者の国籍'
        'number_of_resident':int(s[3]), # 滞在人数
        'days':int(s[1]), #宿泊日数
        
    }
    data.update({'amount':data['number_of_resident']*data['days']})
    start_date = datetime.datetime.strptime(s[2], '%Y/%m/%d')
    # 報告期間を算出
    year = start_date.year-1 if start_date.month < 4 else start_date.year
    start_month = start_date.month//2*2
    end_month = start_month+1 if start_month < 12 else 1
    period = '{}年度{:0>2}月〜{:0>2}月'.format(year, start_month, end_month)
    # 宿泊日の計算
    delta = datetime.timedelta(days=1)
    dates = ';'.join([(start_date+delta*i).strftime('%Y-%m-%d') for i in range(data['days'])])
    data.update({'dates':dates,'period':period})
    #print(data)
    return data
 
def sumup_lines(lines):
    ''' 行データを文字列のリストとして受け取り、統計データをdictで返す
    まずは行ごとの入力を分析して、最終的にはs['期間']['登録番号'] = {}のものを返す
    '''
    summary = {}
    for i in lines:
        record = parse_input(i)
        if record['period'] not in summary:
            summary[record['period']] = {}
        if record['regist_number'] not in summary[record['period']]:
            summary[record['period']][record['regist_number']] = defaultdict(int)
        cur = summary[record['period']][record['regist_number']]
        cur[record['country']] += record['number_of_resident'] # 対応国籍の人数
        cur['number_of_resident'] += record['number_of_resident'] # 合計の人数
        cur['days'] += record['days']
        cur['amount'] += record['amount']
        if 'dates' not in cur:
            cur['dates'] = record['dates']
        else:
            cur['dates'] += ';'+record['dates']
    return summary

def get_format_dict(summary): # -> list
    '''統計されたデータをもとに、formatのkeyに基づいて、フォーマットのdictをlistにして返す
    '''
    all_data = []
    for period in summary:
        for number in summary[period]:
            data = {i[1:-1]:0 for i in line_format.split(',')}
            data['regist_number'] = number
            data['period'] = period
            cur = summary[period][number]
            for key in cur:
                data[key] = cur[key]
            all_data.append(data)
    # ダミーデータを作る
    for period in summary:
        for room in ['101', '102', '201', '202']:
            # この部屋の登録番号がsummaryの中になければ
            if REG_NUM_PAIR[room] not in summary[period]:
                data = {i[1:-1]:0 for i in line_format.split(',')}
                data['regist_number'] = REG_NUM_PAIR[room]
                data['period'] = period
                data['dates'] = ''
                all_data.append(data)
    return all_data

def fill_line(data):
    '''get dict data and fill it into a line format
    '''
    return line_format.format(**data)

def make_report(filepath='./data_to_feed.txt', mode='separate'):
    lines = read_data()
    if not lines:
        if input('レコードがありません、このまま進みますか？(y/N)') != 'y':
            print('Exiting...!')
            return None
        else:
            print('利用者なしのレポート作成します。')
    summary = sumup_lines(lines)
    if not summary:
        year = int(input('Year:'))
        start_month = int(input('Start month:'))
        end_month = start_month+1 if start_month < 12 else 1
        period = '{}年度{:0>2}月〜{:0>2}月'.format(year, start_month, end_month)
        summary[period] = {}
    all_data = get_format_dict(summary)
    
    # 民泊レポートのサイトにバグが有るため、このモードは使えません
    if mode == 'all': # 一つにまとめたレポートを作る
        path = './'+period+'/'
        if not os.path.exists(path):
            os.mkdir(path)
        name = 'all.csv'
        savepath = path+name
        with open(savepath, 'w', encoding='cp932') as csv_out:
            writer = csv.writer(csv_out)
            writer.writerow(csv_head.split(','))
            for data in all_data:
                print(fill_line(data))
                writer.writerow(fill_line(data).split(','))
                
    elif mode == 'separate': # 分割されたレポートを作る
        for data in all_data:
            path = './'+data['period']+'/'
            if not os.path.exists(path):
                os.mkdir(path)
            name = REG_NUM_PAIR[data['regist_number']]+'.csv'
            savepath = path+name
            print(f'{savepath=}')
            with open(savepath, 'w', encoding='shift_jis') as csv_out:
                writer = csv.writer(csv_out)
                for i in [csv_head, fill_line(data)]:
                    print('WRITING--->', i)
                    writer.writerow(i.split(','))
    else: # 引数が変
        print(f'mode arg mast be one of below "all"/"separate"')
        return None


In [4]:
make_report(filepath='./data_to_feed.txt', mode='separate')

レコードがありません、このまま進みますか？(y/N)y
利用者なしのレポート作成します。
Year:2021
Start month:8
savepath='./2021年度08月〜09月/101.csv'
WRITING---> 届出番号,報告期間,宿泊日数,宿泊者数,宿泊延べ人数,日本,韓国,台湾,香港,中国,タイ,シンガポール,マレーシア,インドネシア,フィリピン,ベトナム,インド,英国,ドイツ,フランス,イタリア,スペイン,ロシア,米国,カナダ,オーストラリア,その他,宿泊日
WRITING---> 第M130020507号,2021年度08月〜09月,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
savepath='./2021年度08月〜09月/102.csv'
WRITING---> 届出番号,報告期間,宿泊日数,宿泊者数,宿泊延べ人数,日本,韓国,台湾,香港,中国,タイ,シンガポール,マレーシア,インドネシア,フィリピン,ベトナム,インド,英国,ドイツ,フランス,イタリア,スペイン,ロシア,米国,カナダ,オーストラリア,その他,宿泊日
WRITING---> 第M130020508号,2021年度08月〜09月,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
savepath='./2021年度08月〜09月/201.csv'
WRITING---> 届出番号,報告期間,宿泊日数,宿泊者数,宿泊延べ人数,日本,韓国,台湾,香港,中国,タイ,シンガポール,マレーシア,インドネシア,フィリピン,ベトナム,インド,英国,ドイツ,フランス,イタリア,スペイン,ロシア,米国,カナダ,オーストラリア,その他,宿泊日
WRITING---> 第M130020509号,2021年度08月〜09月,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
savepath='./2021年度08月〜09月/202.csv'
WRITING---> 届出番号,報告期間,宿泊日数,宿泊者数,宿泊延べ人数,日本,韓国,台湾,香港,中国,タイ,シンガポール,マレーシア,インドネシア,フィリピン,ベトナム,インド,英国,ドイツ

# 事業実績報告のCSVファイルについて

1. 詳細仕様は[https://www.mlit.go.jp/common/001225032.pdf](https://www.mlit.go.jp/common/001225032.pdf)ここを参照している。
1. 一回は1番号だけ報告できる
1. UTF-8だと「成功件数0、失敗件数0」になり、全項目チェックしない様になる、ANSIでエンコードすべき
1. 宿泊日の状況はまだわからない

## 知見
- 単純の`f.write('a,b,c,d')`でやると、CSVファイルとして認識されないので、必ずcsv.writerでやる
    - csv.writerのデフォルト設定の一つは`dialect='excel'`、これが重要
    - encoding='shift_jis'も重要
- しかし、CSVファイルを読み取るとき、どういう形で保存されても、データは同じ見た目している
    - つまり、officeで保存されたCSVと、ただのf.write('a,b,c,d')で保存されたCSV、両方読み取ろうとすると、同じ結果になる
- 予測:
    - 恐らく、改行文字が重要になる、もとのやり方では改行文字をUNIXの'\n'を使っている。
    - この改行文字でofficeで読み取れるけど、結局文字コードが違うので、事業報告システムでは、この改行文字を認識できないので、成功0失敗0になる
        - 成功失敗0の理由として、改行文字が認識されてないので、期間の後の部分は読み取られていない、'\n'の後を読み取らず、もとの'excel'式のCSVファイルの改行文字を探そうとしている